# OpenVINO optimizations for Knowledge graphs

The goal of this notebook is to showcase performance optimizations for the ConvE knowledge graph embeddings model using the Intel® Distribution of OpenVINO™ Toolkit. <br>
The optimizations process contains the following steps:

1. Export the trained model to a format suitable for OpenVINO optimizations and inference
2. Report the inference performance speedup obtained with the optimized OpenVINO model

The ConvE model is an implementation of the paper - "Convolutional 2D Knowledge Graph Embeddings" (https://arxiv.org/abs/1707.01476). <br>
The sample dataset can be downloaded from: https://github.com/TimDettmers/ConvE/tree/master/countries/countries_S1
#### Table of contents:
- [Windows specific settings](#Windows-specific-settings)
- [Import the packages needed for successful execution](#Import-the-packages-needed-for-successful-execution)
    - [Settings: Including path to the serialized model files and input data files](#Settings:-Including-path-to-the-serialized-model-files-and-input-data-files)
    - [Download Model Checkpoint](#Download-Model-Checkpoint)
    - [Defining the ConvE model class](#Defining-the-ConvE-model-class)
    - [Defining the dataloader](#Defining-the-dataloader)
    - [Evaluate the trained ConvE model](#Evaluate-the-trained-ConvE-model)
    - [Prediction on the Knowledge graph.](#Prediction-on-the-Knowledge-graph.)
    - [Convert the trained PyTorch model to IR format for OpenVINO inference](#Convert-the-trained-PyTorch-model-to-IR-format-for-OpenVINO-inference)
    - [Evaluate the model performance with OpenVINO](#Evaluate-the-model-performance-with-OpenVINO)
- [Select inference device](#Select-inference-device)
    - [Determine the platform specific speedup obtained through OpenVINO graph optimizations](#Determine-the-platform-specific-speedup-obtained-through-OpenVINO-graph-optimizations)
    - [Benchmark the converted OpenVINO model using benchmark app](#Benchmark-the-converted-OpenVINO-model-using-benchmark-app)
    - [Conclusions](#Conclusions)
    - [References](#References)


In [1]:
%pip install -q "openvino>=2023.1.0"

## Windows specific settings
[back to top ⬆️](#Table-of-contents:)


In [2]:
# On Windows, add the directory that contains cl.exe to the PATH
# to enable PyTorch to find the required C++ tools.
# This code assumes that Visual Studio 2019 is installed in the default directory.
# If you have a different C++ compiler, please add the correct path
# to os.environ["PATH"] directly.
# Note that the C++ Redistributable is not enough to run this notebook.

# Adding the path to os.environ["LIB"] is not always required
# - it depends on the system's configuration

import sys

if sys.platform == "win32":
    import distutils.command.build_ext
    import os
    from pathlib import Path

    VS_INSTALL_DIR = r"C:/Program Files (x86)/Microsoft Visual Studio"
    cl_paths = sorted(list(Path(VS_INSTALL_DIR).glob("**/Hostx86/x64/cl.exe")))
    if len(cl_paths) == 0:
        raise ValueError(
            "Cannot find Visual Studio. This notebook requires a C++ compiler. If you installed "
            "a C++ compiler, please add the directory that contains"
            "cl.exe to `os.environ['PATH']`."
        )
    else:
        # If multiple versions of MSVC are installed, get the most recent version
        cl_path = cl_paths[-1]
        vs_dir = str(cl_path.parent)
        os.environ["PATH"] += f"{os.pathsep}{vs_dir}"
        # Code for finding the library dirs from
        # https://stackoverflow.com/questions/47423246/get-pythons-lib-path
        d = distutils.core.Distribution()
        b = distutils.command.build_ext.build_ext(d)
        b.finalize_options()
        os.environ["LIB"] = os.pathsep.join(b.library_dirs)
        print(f"Added {vs_dir} to PATH")

## Import the packages needed for successful execution
[back to top ⬆️](#Table-of-contents:)


In [3]:
import json
from pathlib import Path
import sys
import time

import numpy as np
import torch
from sklearn.metrics import accuracy_score
from torch.nn import functional as F, Parameter
from torch.nn.init import xavier_normal_

import openvino as ov

# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
);
from notebook_utils import download_file

### Settings: Including path to the serialized model files and input data files
[back to top ⬆️](#Table-of-contents:)



In [4]:
# Path to the pretrained model checkpoint
modelpath = Path('models/conve.pt')

# Entity and relation embedding dimensions
EMB_DIM = 300

# Top K vals to consider from the predictions
TOP_K = 2

# Required for OpenVINO conversion
output_dir = Path("models")
base_model_name = "conve"

output_dir.mkdir(exist_ok=True)

# Paths where PyTorch and OpenVINO IR models will be stored
ir_path = Path(output_dir / base_model_name).with_suffix(".xml")

In [5]:
data_folder = "data"

# Download the file containing the entities and entity IDs
entdatapath = download_file(
    "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/text/countries_S1/kg_training_entids.txt",
    directory=data_folder
)

# Download the file containing the relations and relation IDs
reldatapath = download_file(
    "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/text/countries_S1/kg_training_relids.txt",
    directory=data_folder
)

# Download the test data file
testdatapath = download_file(
    "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/json/countries_S1/e1rel_to_e2_ranking_test.json",
    directory=data_folder
)

data/kg_training_entids.txt:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

data/kg_training_relids.txt:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

data/e1rel_to_e2_ranking_test.json:   0%|          | 0.00/19.1k [00:00<?, ?B/s]

### Download Model Checkpoint
[back to top ⬆️](#Table-of-contents:)


In [6]:
model_url = 'https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/knowledge-graph-embeddings/conve.pt'

download_file(model_url, filename=modelpath.name, directory=modelpath.parent)

'models/conve.pt' already exists.


PosixPath('/home/ea/work/openvino_notebooks/notebooks/219-knowledge-graphs-conve/models/conve.pt')

### Defining the ConvE model class
[back to top ⬆️](#Table-of-contents:)


In [7]:
# Model implementation reference: https://github.com/TimDettmers/ConvE
class ConvE(torch.nn.Module):
    def __init__(self, num_entities, num_relations, emb_dim):
        super(ConvE, self).__init__()
        # Embedding tables for entity and relations with num_uniq_ent in y-dim, emb_dim in x-dim
        self.emb_e = torch.nn.Embedding(num_entities, emb_dim, padding_idx=0)
        self.ent_weights_matrix = torch.ones([num_entities, emb_dim], dtype=torch.float64)
        self.emb_rel = torch.nn.Embedding(num_relations, emb_dim, padding_idx=0)
        self.ne = num_entities
        self.nr = num_relations
        self.inp_drop = torch.nn.Dropout(0.2)
        self.hidden_drop = torch.nn.Dropout(0.3)
        self.feature_map_drop = torch.nn.Dropout2d(0.2)
        self.loss = torch.nn.BCELoss()
        self.conv1 = torch.nn.Conv2d(1, 32, (3, 3), 1, 0, bias=True)
        self.bn0 = torch.nn.BatchNorm2d(1)
        self.bn1 = torch.nn.BatchNorm2d(32)
        self.ln0 = torch.nn.LayerNorm(emb_dim)
        self.register_parameter('b', Parameter(torch.zeros(num_entities)))
        self.fc = torch.nn.Linear(16128, emb_dim)

    def init(self):
        """ Initializes the model """
        # Xavier initialization
        xavier_normal_(self.emb_e.weight.data)
        xavier_normal_(self.emb_rel.weight.data)

    def forward(self, e1, rel):
        """ Forward pass on the model.
        :param e1: source entity
        :param rel: relation between the source and target entities
        Returns the model predictions for the target entities
        """
        e1_embedded = self.emb_e(e1).view(-1, 1, 10, 30)
        rel_embedded = self.emb_rel(rel).view(-1, 1, 10, 30)
        stacked_inputs = torch.cat([e1_embedded, rel_embedded], 2)
        stacked_inputs = self.bn0(stacked_inputs)
        x = self.inp_drop(stacked_inputs)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.feature_map_drop(x)
        x = x.view(1, -1)
        x = self.fc(x)
        x = self.hidden_drop(x)
        x = self.ln0(x)
        x = F.relu(x)
        x = torch.mm(x, self.emb_e.weight.transpose(1, 0))
        x = self.hidden_drop(x)
        x += self.b.expand_as(x)
        pred = torch.nn.functional.softmax(x, dim=1)
        return pred

### Defining the dataloader
[back to top ⬆️](#Table-of-contents:)


In [8]:
class DataLoader():
    def __init__(self):
        super(DataLoader, self).__init__()

        self.ent_path = entdatapath
        self.rel_path = reldatapath
        self.test_file = testdatapath
        self.entity_ids, self.ids2entities = self.load_data(data_path=self.ent_path)
        self.rel_ids, self.ids2rel = self.load_data(data_path=self.rel_path)
        self.test_triples_list = self.convert_triples(data_path=self.test_file)

    def load_data(self, data_path):
        """ Creates a dictionary of data items with corresponding ids """
        item_dict, ids_dict = {}, {}
        fp = open(data_path, "r")
        lines = fp.readlines()
        for line in lines:
            name, id = line.strip().split('\t')
            item_dict[name] = int(id)
            ids_dict[int(id)] = name
        fp.close()
        return item_dict, ids_dict

    def convert_triples(self, data_path):
        """ Creates a triple of source entity, relation and target entities"""
        triples_list = []
        dp = open(data_path, "r")
        lines = dp.readlines()
        for line in lines:
            item_dict = json.loads(line.strip())
            h = item_dict['e1']
            r = item_dict['rel']
            t = item_dict['e2_multi1'].split('\t')
            hrt_list = []
            hrt_list.append(self.entity_ids[h])
            hrt_list.append(self.rel_ids[r])
            t_ents = []
            for t_idx in t:
                t_ents.append(self.entity_ids[t_idx])
            hrt_list.append(t_ents)
            triples_list.append(hrt_list)
        dp.close()
        return triples_list

### Evaluate the trained ConvE model
[back to top ⬆️](#Table-of-contents:)

First, we will evaluate the model performance using PyTorch. The goal is to make sure there are no accuracy differences between the original model inference and the model converted to OpenVINO intermediate representation inference results. <br>
Here, we use a simple accuracy metric to evaluate the model performance on a test dataset. However, it is typical to use metrics such as Mean Reciprocal Rank, Hits@10 etc.

In [9]:
data = DataLoader()
num_entities = len(data.entity_ids)
num_relations = len(data.rel_ids)

model = ConvE(num_entities=num_entities, num_relations=num_relations, emb_dim=EMB_DIM)
model.load_state_dict(torch.load(modelpath))
model.eval()

pt_inf_times = []

triples_list = data.test_triples_list
num_test_samples = len(triples_list)
pt_acc = 0.0
for i in range(num_test_samples):
    test_sample = triples_list[i]
    h, r, t = test_sample
    start_time = time.time()
    logits = model.forward(e1=torch.tensor(h), rel=torch.tensor(r))
    end_time = time.time()
    pt_inf_times.append(end_time - start_time)
    score, pred = torch.topk(logits, TOP_K, 1)

    gt = np.array(sorted(t))
    pred = np.array(sorted(pred[0].cpu().detach()))
    pt_acc += accuracy_score(gt, pred)

avg_pt_time = np.mean(pt_inf_times) * 1000
print(f'Average time taken for inference: {avg_pt_time} ms')
print(f'Mean accuracy of the model on the test dataset: {pt_acc/num_test_samples}')

Average time taken for inference: 0.8238554000854492 ms
Mean accuracy of the model on the test dataset: 0.875


### Prediction on the Knowledge graph.
[back to top ⬆️](#Table-of-contents:)

Here, we perform the entity prediction on the knowledge graph, as a sample evaluation task. <br>
We pass the source entity `san_marino` and relation `locatedIn` to the knowledge graph and obtain the target entity predictions. <br>
Expected predictions are target entities that form a factual triple with the entity and relation passed as inputs to the knowledge graph.

In [10]:
entitynames_dict = data.ids2entities

ent = 'san_marino'
rel = 'locatedin'

h_idx = data.entity_ids[ent]
r_idx = data.rel_ids[rel]

logits = model.forward(torch.tensor(h_idx), torch.tensor(r_idx))
score, pred = torch.topk(logits, TOP_K, 1)

for j, id in enumerate(pred[0].cpu().detach().numpy()):
    pred_entity = entitynames_dict[id]
    print(f'Source Entity: {ent}, Relation: {rel}, Target entity prediction: {pred_entity}')

Source Entity: san_marino, Relation: locatedin, Target entity prediction: southern_europe
Source Entity: san_marino, Relation: locatedin, Target entity prediction: europe


### Convert the trained PyTorch model to IR format for OpenVINO inference
[back to top ⬆️](#Table-of-contents:)

To evaluate performance with OpenVINO, we can either convert the trained PyTorch model to an intermediate representation (IR) format. `ov.convert_model` function can be used for conversion PyTorch models to OpenVINO Model class instance, that is ready to load on device or can be saved on disk in OpenVINO Intermediate Representation (IR) format using `ov.save_model`.

In [11]:
print('Converting the trained conve model to IR format')

ov_model = ov.convert_model(model, example_input=(torch.tensor(1), torch.tensor(1)))
ov.save_model(ov_model, ir_path)

Converting the trained conve model to IR format


### Evaluate the model performance with OpenVINO
[back to top ⬆️](#Table-of-contents:)

Now, we evaluate the model performance with the OpenVINO framework. In order to do so, make three main API calls:

1. Initialize the Inference engine with `Core()`
2. Load the model with `read_model()`
3. Compile the model with `compile_model()`

Then, the model can be inferred on by using the `create_infer_request()` API call.

In [12]:
core = ov.Core()
ov_model = core.read_model(model=ir_path)

## Select inference device
[back to top ⬆️](#Table-of-contents:)

select device from dropdown list for running inference using OpenVINO

In [13]:
import ipywidgets as widgets

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='CPU',
    description='Device:',
    disabled=False,
)

device

Dropdown(description='Device:', options=('CPU', 'GPU.0', 'GPU.1', 'AUTO'), value='CPU')

In [14]:
compiled_model = core.compile_model(model=ov_model, device_name=device.value)
input_layer_source = compiled_model.inputs[0]
input_layer_relation = compiled_model.inputs[1]
output_layer = compiled_model.output(0)

ov_acc = 0.0
ov_inf_times = []
for i in range(num_test_samples):
    test_sample = triples_list[i]
    source, relation, target = test_sample
    model_inputs = {input_layer_source: np.int64(source), input_layer_relation: np.int64(relation)}
    start_time = time.time()
    result = compiled_model(model_inputs)[output_layer]
    end_time = time.time()
    ov_inf_times.append(end_time - start_time)
    top_k_idxs = list(np.argpartition(result[0], -TOP_K)[-TOP_K:])

    gt = np.array(sorted(t))
    pred = np.array(sorted(top_k_idxs))
    ov_acc += accuracy_score(gt, pred)

avg_ov_time = np.mean(ov_inf_times) * 1000
print(f'Average time taken for inference: {avg_ov_time} ms')
print(f'Mean accuracy of the model on the test dataset: {ov_acc/num_test_samples}')

Average time taken for inference: 0.7051527500152588 ms
Mean accuracy of the model on the test dataset: 0.10416666666666667


### Determine the platform specific speedup obtained through OpenVINO graph optimizations
[back to top ⬆️](#Table-of-contents:)


In [15]:
# prevent division by zero
delimiter = max(avg_ov_time, np.finfo(float).eps)

print(f'Speedup with OpenVINO optimizations: {round(float(avg_pt_time)/float(delimiter),2)} X')

Speedup with OpenVINO optimizations: 1.17 X


### Benchmark the converted OpenVINO model using benchmark app
[back to top ⬆️](#Table-of-contents:)

The OpenVINO toolkit provides a benchmarking application to gauge the platform specific runtime performance that can be obtained under optimal configuration parameters for a given model. For more details refer to: https://docs.openvino.ai/2023.3/openvino_sample_benchmark_tool.html

Here, we use the benchmark application to obtain performance estimates under optimal configuration for the knowledge graph model inference. <br> 
We obtain the average (AVG), minimum (MIN) as well as maximum (MAX) latency as well as the throughput performance (in samples/s) observed while running the benchmark application. <br> 
The platform specific optimal configuration parameters determined by the benchmarking app for OpenVINO inference can also be obtained by looking at the benchmark app results. 

In [16]:
print('Benchmark OpenVINO model using the benchmark app')
! benchmark_app -m $ir_path -d $device.value -api async -t 10 -shape "input.1[1],input.2[1]" 

Benchmark OpenVINO model using the benchmark app
[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.0.0-13770-9b52171d290
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.0.0-13770-9b52171d290
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to PerformanceMode.THROUGHPUT.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 8.35 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     e1 (node: e1) : i64 / [...] / []
[ INFO ]     rel (node: rel) : i64 / [...] / []
[ INFO ] Model outputs:
[ INFO ]     ***NO_NAME*** (node: aten::softmax/Softmax) : f32 / [...] / [1,271]
[Step 5/11] Resizing model to match image sizes a

### Conclusions
[back to top ⬆️](#Table-of-contents:)

In this notebook, we convert the trained PyTorch knowledge graph embeddings model to the OpenVINO format. <br>
We confirm that there are no accuracy differences post conversion. We also perform a sample evaluation on the knowledge graph.<br> 
Then, we determine the platform specific speedup in runtime performance that can be obtained through OpenVINO graph optimizations. <br>
To learn more about the OpenVINO performance optimizations, refer to: https://docs.openvino.ai/2023.3/openvino_docs_deployment_optimization_guide_dldt_optimization_guide.html 

### References
[back to top ⬆️](#Table-of-contents:)

1. Convolutional 2D Knowledge Graph Embeddings, Tim Dettmers et al. (https://arxiv.org/abs/1707.01476)
2. Model implementation: https://github.com/TimDettmers/ConvE 

The ConvE model implementation used in this notebook is licensed under the MIT License. The license is displayed below: <br>
MIT License

Copyright (c) 2017 Tim Dettmers

Permission is hereby granted, free of charge, to any person obtaining a copy<br>
of this software and associated documentation files (the "Software"), to deal<br>
in the Software without restriction, including without limitation the rights<br>
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell<br>
copies of the Software, and to permit persons to whom the Software is<br>
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all<br>
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR<br>
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,<br>
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE<br>
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER<br>
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,<br>
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE<br>
SOFTWARE.